<a href="https://colab.research.google.com/github/catalin-giuglan/BoneFractureDetection/blob/main/ProiectCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import os

model = YOLO('/content/drive/MyDrive/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures/weights/best.pt')

base_results_dir = '/content/drive/MyDrive/Proiect_CDv2/Proiect/validation_results'

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='train', save_dir=os.path.join(base_results_dir, 'train_results'))

print(" REZULTATE FINALE - TRAIN SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='test', save_dir=os.path.join(base_results_dir, 'test_results'))

print("\n REZULTATE FINALE - TEST SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='val', save_dir=os.path.join(base_results_dir, 'val_results'))
print("\n REZULTATE FINALE - VALID SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import glob
from ultralytics import YOLO

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
DATA_PATH = "/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml"
TRAIN_IMAGES_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/train/images"
PROJECT_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/runs"
RUN_NAME = "YOLOv8n_SimCLR_Fractures_6"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class XRayContrastiveDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_paths = glob.glob(os.path.join(img_dir, "*.jpg")) + glob.glob(os.path.join(img_dir, "*.png"))
        self.transform = transform
    def __len__(self): return len(self.img_paths)
    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        return self.transform(img), self.transform(img)

class SimCLR_YOLO_Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        base = YOLO("yolov8n.pt").model
        self.backbone = nn.Sequential(*list(base.children())[0][:10])
        self.projection_head = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
            nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 64)
        )
    def forward(self, x):
        return self.projection_head(self.backbone(x))

def nt_xent_loss(z1, z2, temp=0.1):
    z1, z2 = nn.functional.normalize(z1), nn.functional.normalize(z2)
    logits = torch.matmul(z1, z2.T) / temp
    labels = torch.arange(z1.size(0)).to(z1.device)
    return nn.functional.cross_entropy(logits, labels)

print("--- START SIMCLR PRE-TRAINING (Self-Supervised) ---")
aug = transforms.Compose([
    transforms.RandomResizedCrop(512),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

dataset = XRayContrastiveDataset(TRAIN_IMAGES_DIR, aug)
loader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
simclr_model = SimCLR_YOLO_Backbone().to(DEVICE)
optimizer = torch.optim.Adam(simclr_model.parameters(), lr=1e-3)

simclr_model.train()
for epoch in range(60):
    total_loss = 0
    for (img1, img2) in loader:
        optimizer.zero_grad()
        z1, z2 = simclr_model(img1.to(DEVICE)), simclr_model(img2.to(DEVICE))
        loss = nt_xent_loss(z1, z2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

torch.save(simclr_model.backbone.state_dict(), "temp_backbone.pth")
print("SimCLR Pre-training done. Weights saved.")

print("\n--- START YOLOv8 TRAINING (Supervised) ---")
model = YOLO("yolov8n.pt")

backbone_module = model.model.model[:10]

print("Loading SimCLR weights into backbone...")
backbone_module.load_state_dict(torch.load("temp_backbone.pth"), strict=False)

results = model.train(
    data=DATA_PATH,
    project=PROJECT_DIR,
    name=RUN_NAME,
    imgsz=256,
    batch=512,
    epochs=150,
    patience=30,
    optimizer="auto",
    mosaic=0.0,
    mixup=0.05,
    label_smoothing=0.05
)

val_results = model.val()
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")
model.export(format="onnx")

In [ ]:
from ultralytics import YOLO
import os

model = YOLO('/content/drive/MyDrive/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures/weights/best.pt')

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='train')

print(" REZULTATE FINALE - TRAIN SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='test')

print("\n REZULTATE FINALE - TEST SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

result = model.val(data='/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml', split='val')
print("\n REZULTATE FINALE - VALID SET ")
print(f"mAP50: {result.box.map50:.4f}")
print(f"mAP50-95: {result.box.map:.4f}")
print(f"Precision: {result.box.mp:.4f}")
print(f"Recall: {result.box.mr:.4f}")

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import glob
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/Proiect_CDv2/Proiect/runs/YOLOv8n_SimCLR_Fractures/weights/best.pt")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
DATA_PATH = "/content/drive/MyDrive/Proiect_CDv2/Proiect/data.yaml"
TRAIN_IMAGES_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/train/images"
PROJECT_DIR = "/content/drive/MyDrive/Proiect_CDv2/Proiect/runs"
RUN_NAME = "YOLOv8n_SimCLR_Fractures_512batch&256size_12"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results = model.train(
    data=DATA_PATH,
    project=PROJECT_DIR,
    name=RUN_NAME,
    imgsz=256,
    batch=512,
    epochs=150,
    patience=30,
    optimizer="auto",
    mosaic=0.0,
    mixup=0.05,
    label_smoothing=0.05
)

val_results = model.val()
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")
model.export(format="onnx")